In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/PhoBERT')

Mounted at /content/drive/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.7 MB/s eta 0:00:00


In [ ]:
!pip install vncorenlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645933 sha256=5f05bc8c5a9b22ccc977f76bbcfe2f08d5cb06df5be2a232344e9e1f471b3848
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp


In [ ]:
!pip install fastBPE
!pip install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp310-cp310-linux_x86_64.whl size=767963 sha256=6eaefa03be311bc6edbced5fe1518620c6fbb2e2f51e79b49d899892fdef8d2a
  Stored in directory: /root/.cache/pip/wheels/13/5d/b9/4b8897941ebc9e8c6cc3f3ffd3ea5115731754269205098754
Successfully built fastBPE
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 57.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/PhoBERT/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

In [ ]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes',
    default="/content/drive/MyDrive/PhoBERT/PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("/content/drive/MyDrive/PhoBERT/PhoBERT_base_transformers/dict.txt")

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_sentence(sentence):
  MAX_LEN = 125
  test_text, test_ids, test_masks = [], [], []
  splits = sentence.strip().split('\n')
  text = ' '.join(splits)
  text = rdrsegmenter.tokenize(text)
  text = ' '.join([' '.join(x) for x in text])
  test_text.append(text)
  MAX_LEN = 125
  for sent in test_text:
      subwords = '<s> ' + bpe.encode(sent) + ' </s>'
      encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
      test_ids.append(encoded_sent)
  test_ids = pad_sequences(test_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
  for sent in test_ids:
    mask = [int(token_id > 0) for token_id in sent]
    test_masks.append(mask)
  return test_ids, test_masks

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch

# Đường dẫn tới thư mục chứa model đã lưu
model_path = "/content/drive/MyDrive/PhoBERT/pretrained/pytorch_model.bin"

# Load lại model
config = RobertaConfig.from_pretrained('/content/drive/MyDrive/PhoBERT/pretrained/config.json')
model = RobertaForSequenceClassification.from_pretrained(model_path, config=config)

label_mapping = {0: 'Tích cực', 1: 'Tiêu cực'}
# Hàm dự đoán
def predict_sentence(sentence):
    # Tiền xử lý câu đầu vào (tương tự như quá trình tiền xử lý cho dữ liệu train và test)
    input_ids, input_masks = preprocess_sentence(sentence)

    # Chuyển đổi thành Tensor
    test_inputs = torch.tensor(input_ids)
    test_masks = torch.tensor(input_masks)

    # Dự đoán
    model.eval()
    with torch.no_grad():
        outputs = model(test_inputs, attention_mask=test_masks)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()

    # Chuyển đổi nhãn dự đoán thành chữ cái
    predicted_labels = [label_mapping[label] for label in predicted_labels]

    # Trả về nhãn dự đoán
    return predicted_labels[0]

In [ ]:
user_sentence = "Giao hàng lâu. Sai màu . Nhanh trôi. Hóng mãi cuối cùng quá nản"

predicted_label = predict_sentence(user_sentence)

print("Dự đoán:", predicted_label)

Dự đoán: Tiêu cực
